In [11]:
import chromadb
from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

# Initialize ChromaDB
client = chromadb.PersistentClient(path="./research_db")
collection = client.get_or_create_collection(
    name="ml_publications",
    metadata={"hnsw:space": "cosine"}
)

# Set up our embedding model
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2")

RuntimeError: Failed to import transformers.modeling_utils because of the following error (look up to see its traceback):
module 'torch' has no attribute 'compiler'

In [ ]:
import os
from langchain_community.document_loaders import PDFLoader

def load_research_publications(documents_path):
    """Loading all the available pdf files in the data directory"""
    
    # List to store all documents
    documents = []
    
    # Load each .txt file in the documents folder
    for file in os.listdir(documents_path):
        if file.endswith(".pdf"):
            file_path = os.path.join(documents_path, file)
            try:
                loader = PDFLoader(file_path)
                loaded_docs = loader.load()
                documents.extend(loaded_docs)
                print(f"Successfully loaded: {file}")
            except Exception as e:
                print(f"Error loading {file}: {str(e)}")
    
    print(f"\nTotal documents loaded: {len(documents)}")
    
    # Extract content as strings and return
    publications = []
    for doc in documents:
        publications.append(doc.page_content)
    
    return publications

In [ ]:
def chunk_research_paper(paper_content, title):
    """Break a research paper into searchable chunks"""
    
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,          # ~200 words per chunk
        chunk_overlap=200,        # Overlap to preserve context
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    
    chunks = text_splitter.split_text(paper_content)
    
    # Add metadata to each chunk
    chunk_data = []
    for i, chunk in enumerate(chunks):
        chunk_data.append({
            "content": chunk,
            "title": title,
            "chunk_id": f"{title}_{i}",
        })
    
    return chunk_data

In [ ]:
import torch
from langchain_huggingface import HuggingFaceEmbeddings

def embed_documents(documents: list[str]) -> list[list[float]]:
    """
    Embed documents using a model.
    """
    device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps" if torch.backends.mps.is_available() else "cpu"
    )
    model = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={"device": device},
    )

    embeddings = model.embed_documents(documents)
    return embeddings

In [ ]:
def insert_publications(collection: chromadb.Collection, publications: list[str]):
    """
    Insert documents into a ChromaDB collection.

    Args:
        collection (chromadb.Collection): The collection to insert documents into
        publications (list[str]): The documents to insert

    Returns:
        None
    """
    next_id = collection.count()

    for publication in publications:
        chunked_publication = chunk_publication(publication)
        embeddings = embed_documents(chunked_publication)
        ids = list(range(next_id, next_id + len(chunked_publication)))
        ids = [f"document_{id}" for id in ids]
        collection.add(
            embeddings=embeddings,
            ids=ids,
            documents=chunked_publication,
        )
        next_id += len(chunked_publication)

In [ ]:
def search_research_db(query, collection, embeddings, top_k=5):
    """Find the most relevant research chunks for a query"""
    
    # Convert question to vector
    query_vector = embeddings.embed_query(query)
    
    # Search for similar content
    results = collection.query(
        query_embeddings=[query_vector],
        n_results=top_k,
        include=["documents", "metadatas", "distances"]
    )
    
    # Format results
    relevant_chunks = []
    for i, doc in enumerate(results["documents"][0]):
        relevant_chunks.append({
            "content": doc,
            "title": results["metadatas"][0][i]["title"],
            "similarity": 1 - results["distances"][0][i]  # Convert distance to similarity
        })
    
    return relevant_chunks

In [ ]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate

def answer_research_question(topic, collection, embeddings, llm):
    """Generate an answer based on retrieved research"""
    
    # Get relevant research chunks
    relevant_chunks = search_research_db(topic, collection, embeddings, top_k=3)
    
    # Build context from research
    context = "\n\n".join([
        f"From {chunk['title']}:\n{chunk['content']}" 
        for chunk in relevant_chunks
    ])
    
    # Prompt for question generation
    question_template = PromptTemplate(
        input_variables=["topic"],
        template="""
        You are an AI professional assistant helping students evaluate their knowledge gaps.
        
        Your task is as follows:
        Generate 5 closed questions and 5 open questions on the topic {topic}

        Ensure your response follows these rules:
        - Each question should be approximately 70 words.
        - Each question should target the underlying nuances of the topic.

        Your goal is to provide sets of questions that serve as an evaluation benchmark.

        Style guide:
        - Return only the questions.
        - Use <endofquestion> as delimiter to separate questions.
        """
    )
    
    # Prompt for answer evaluation
    evaluating_template = PromptTemplate(
        input_variables=["question", "answer", "context"], 
        template="""
        Based on the following research findings, assess how accurate the answer is to the question.

        Research Context:
        {context}

        Question: {question}

        Answer: {answer}

        Your response format:
        - Return an accuracy score in 2 decimal places followed by knowledge gaps.
        - Use bullet points to:
          - Highlight brilliant points (if any)
          - List what should have been included
          - Point out irrelevant or incorrect parts
          - Mention common pitfalls to avoid

        Use clear language over jargon. Response should not exceed 150 words.
        """
    )
    
    # Build chains
    question_chain = question_template | llm
    evaluating_chain = evaluating_template | llm

    # Generate questions
    response = question_chain.invoke({'topic': topic})
    questions = response.content.strip().split('<endofquestion>')
    questions = [q.strip() for q in questions if q.strip()]
    
    final_response = []
    
    for i, question in enumerate(questions, start=1):
        print(f"\nQuestion {i}:\n{question}\n")
        answer = input("Provide your answer for the question above:\n")
        evaluation = evaluating_chain.invoke({
            'question': question,
            'answer': answer,
            'context': context
        })
        final_response.append({
            'question': question,
            'answer': answer,
            'evaluation': evaluation.content.strip()
        })
        
    return final_response, relevant_chunks


# Initialize LLM and get answer
llm = ChatGroq(model="llama3-8b-8192")
answer, sources = answer_research_question(
    "Programming language design",
    collection, 
    embeddings, 
    llm
)

print("AI Answer:", answer)
print("\nBased on sources:")
for source in sources:
    print(f"- {source['title']}")